# Data Journalism - COVID-19
Ein Projekt zur Erbringungen der portfoliorelevanten Leistung im Kurs **Data Journalism**:  
Modul: **23-TXT-BaCL5**  
Studiengang: **Texttechnologie und Computerlinguistik**  
Universität: **Universität Bielefeld**  


Vorgelegt von **[Fabian Wohlgemuth](https://www.fabianwohlgemuth.de)**.

----

# Vorraussetzungen

Alle Programme, die zur Ausführung des Codes notwendig sind, befinden sich in der Textdatei `requirements.txt` und lassen sich über den folgenden Befehl per `pip3` installieren.

`pip3 install -r requirements.txt`

# Pakete

Für die Ausführung des Projektes benötigen wir folgende Python-Pakete:

- `pandas`, um den Typ `DataFrame` zu nutzen und unsere tabellarischen Daten zu verarbeiten.
- `plotly`, um die Daten auf einer Landkarte darzustellen.
- `urllib.requests`, um die aktuellsten Zahlen herunterzuladen.
- `datetime`, um das heutige Datum herauszufinden.

In [1]:
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import urllib.request
from datetime import datetime, timedelta

# Datensatz

## Quelle

Der Datensatz ist den Zahlen des **ECDC - European Centre for Disease Prevention and Control** ([Website](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide)) entnommmen.

## Automatisierte Beschaffung

Die Daten befinden sich auf der Website des ECDC im `.xlsx`-Format. Zunächst wird die Datei heruntergeladen und im `assets/`-Ordner gesichert. Dabei versuchen wir, die Daten von Heute herunterzuladen. Sollten diese (noch) nicht verfügbar sein, werden die Daten vom gestrigen Tag heruntergeladen. Wenn auch diese nicht verfügbar sind, findet ein Fallback statt auf die letzte händisch heruntergeladene und geprüfte Datei vom `21.03.2020`.

Nach dem Versuch, die Tagesaktuellen Zahlen herunterzuladen, wird die Variable `data_file` auf die aktuellste Datei festgelegt.

In [40]:
url_base = 'https://www.ecdc.europa.eu/sites/default/files/documents/'
url_file = 'COVID-19-geographic-disbtribution-worldwide-{}.xlsx'

try:
    date = datetime.date(datetime.now())
    url_today = url_file.format(date)
    url = url_base + url_today
    
    urllib.request.urlretrieve(url, 'assets/' + url_today)
    url_file = url_today
except:
    try:
        date = date - timedelta(days=1)
        url_yesterday = url_file.format(date)
        url = url_base + url_yesterday

        urllib.request.urlretrieve(url, 'assets/' + url_yesterday)
        url_file = url_yesterday
    except:
        url_file = 'COVID-19-geographic-disbtribution-worldwide-2020-03-21.xlsx'
finally:
    data_file = url_file
    print('Genutzte Datei: ' + data_file)

Genutzte Datei: COVID-19-geographic-disbtribution-worldwide-2020-03-24.xlsx


Im Anschluss wird die Datei über `pandas` eingelesen und als `DataFrame`-Objekt gespeichert, damit wir die Daten tabellarisch auswerten können. Bevor wir Änderungen an den Daten vornehmen, werden wir diese in einer weiteren Variable zwischenspeichern. Auf diese Art und Weise können wir auch nach Veränderungen immer auf die Ausgangsdaten zurückgreifen, um etwaige Fehler zu finden.

Zum Test der Funktionalität, lassen wir uns die ersten Zeilen des entstandenen `DataFrame` ausgeben. Dies dient außerdem der Überprüfung, ob die Daten weiterer Bereinigung bedürfen.

In [41]:
data = pd.read_excel('assets/' + data_file)
data_raw = data
data.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId
0,2020-03-24,24,3,2020,6,1,Afghanistan,AF
1,2020-03-23,23,3,2020,10,0,Afghanistan,AF
2,2020-03-22,22,3,2020,0,0,Afghanistan,AF
3,2020-03-21,21,3,2020,2,0,Afghanistan,AF
4,2020-03-20,20,3,2020,0,0,Afghanistan,AF


## Sichtung

Die ersten Zeilen der Datei benötigen keine Bereinigung. Sowohl die Indizes als auch die Kopfzeile funktionieren wie gewünscht. Da wir eine Datumsspalte haben, werden die Spalten für Tag, Monat und Jahr prinzipiell nicht benötigt. Da diese im weiteren Verlauf jedoch nicht störend sind, können wir die Spalten so beibehalten.

Eine Fehlerquelle in den Daten kann eine fehlende oder falsch formatierte `GeoId` sein, weshalb wir uns die vorhandenen `GeoId`s ausgeben lassen. Da wir keine Duplikate haben wollen, nutzen wir für die Ausgabe eine Menge.

In [42]:
geoids = set()

for geoid in data['GeoId']:
    geoids.add(geoid)

print('Anzahl der verschiedenen Länder: ' + str(len(geoids)))
print(geoids)

Anzahl der verschiedenen Länder: 185
{nan, 'DE', 'JPG11668', 'MA', 'IE', 'AZ', 'FJ', 'GY', 'SR', 'NC', 'PYF', 'ER', 'KR', 'RO', 'LV', 'SN', 'JE', 'CV', 'FR', 'MM', 'TZ', 'MD', 'BS', 'CD', 'DO', 'GR', 'XK', 'PH', 'BH', 'NG', 'CA', 'TH', 'US', 'BE', 'CM', 'PL', 'AD', 'BT', 'SA', 'BB', 'TR', 'GI', 'BF', 'HN', 'PT', 'PS', 'CU', 'VN', 'TW', 'ET', 'CI', 'AN', 'FO', 'IM', 'CH', 'JP', 'IT', 'GU', 'CL', 'DK', 'DJ', 'BN', 'BD', 'TG', 'MC', 'VE', 'AE', 'UZ', 'GH', 'CG', 'NI', 'BY', 'GM', 'ME', 'SK', 'HU', 'PE', 'AM', 'MY', 'FI', 'SI', 'AO', 'GG', 'SE', 'TN', 'RW', 'VI', 'DM', 'VA', 'BR', 'TL', 'MR', 'DZ', 'AG', 'PY', 'OM', 'BZ', 'IS', 'LT', 'BA', 'ZW', 'SV', 'LI', 'LK', 'TD', 'LB', 'HT', 'PK', 'BO', 'UG', 'RS', 'GE', 'MN', 'UY', 'MG', 'NL', 'MT', 'BJ', 'MZ', 'KY', 'MU', 'CY', 'GL', 'IL', 'JO', 'MV', 'KW', 'NP', 'QA', 'BM', 'NZ', 'EG', 'LR', 'VC', 'UA', 'JM', 'AU', 'SO', 'HR', 'GT', 'SG', 'ZM', 'AT', 'CO', 'CN', 'AL', 'ES', 'CF', 'GA', 'PA', 'EC', 'UK', 'MS', 'ID', 'EE', 'RU', 'MK', 'LC', 'SZ', 'L

Hier fällt auf, dass einige Einträge vorhanden sind, die nicht über einen zweistelligen Länder-Code abgebildet werden. Die nicht-zweistelligen Ländercodes ermitteln wir wiefolgt, um herauszufinden, bei welchen Einträgen Reinigungsbedarf besteht.

In [43]:
geoid_error = data[data['GeoId'].str.len() != 2]

false_geoid = set()

for country in geoid_error['Countries and territories']:
    false_geoid.add(country)
    
for false_id in false_geoid:
    print(false_id)

French_Polynesia
Cases_on_an_international_conveyance_Japan
Namibia


Wirft man einen Blick auf diese Einträge, stellt man fest, dass für *Namibia* keine Einträge in der `GeoId` vorhanden sind (`nan`), *French_Polynesia* bereits einen dreistelligen Länder-Code eingetragen hat und *Cases_on_an_international_conveyance_Japan* eine spezielle achtstellige `GeoId` zugewiesen bekommen hat.

Über eine kurze Recherche lässt sich schnell herausfinden, dass der `Alpha-3`-Code für *Namibia* `NAM` ist. Bei *Cases_on_an_international_conveyance_Japan* handelt es sich um das Passagier-Schiff *Diamond Princess*, welches vor dem Hafen von Yokohama in Japan liegt/lag und in den Daten nicht zu Japans Fällen dazugezählt wird.

An dieser Stelle haben wir zwei Möglichkeiten, die Daten zu bereinigen, da `Plotly` zur Darstellung der Daten dreistellige Länder-Codes benötigt.

1. Die Fälle der *Diamond Princess* nicht auf der Weltkarte darstellen, also ein Sub-Set unserer Daten erstellen, aus dem wir diese herausnehmen
2. Die Fälle einem Land (z.B. Japan) zuordnen
3. Die Fälle weiterhin unter *Cases_on_an_international_conveyance_Japan* führen

Für das Erstellen der Weltkarte benötigen wir zwingend dreistellige Länder-Codes. Da die *Diamond Princess* nicht über einen solchen verfügt, ist für die Weltkarte die dritte Option nicht nützlich. Dennoch werden wir die Daten erhalten, um über Gesamt-Fälle Aussagen treffen zu können. Diese Daten werden weiter unter der Variable `data` geführt. Die Informationen zu den Einträgen, die keinen gültigen dreistelligen Länder-Code enthalten, werden entsprechend auf Weltkarten dann nicht dargestellt.


## Aufbereitung & Bereinigung

Für Namibia können wir bereits den entsprechenden Länder-Code in die `GeoId`-Spalte einfügen.

In [44]:
data.replace("nan", "NA", inplace=True)

Um die Daten im späteren Verlauf per `plotly` auf einer Weltkarte darstellen zu können, benötigen wir Länder-Codes im Format `ISO3166 Alpha-3`. Die `GeoId` aus den vorhanden Daten nutzt jedoch `ISO3166 Alpha-2`, weshalb wir eine weitere Spalte zu unseren Daten hinzufügen werden, die die entsprechenden Codes enthält. Hier bedienen wir uns einer Liste, die sowohl `Alpha-2`- als auch `Alpha-3`-Codes enthält.

In [45]:
iso3166 = pd.read_csv('assets/iso3166.csv')
iso3166.head()

,ISO3166-ALPHA-2,ISO3166-ALPHA-3
0,AF,AFG
1,AX,ALA
2,AL,ALB
3,AN,ANT
4,DZ,DZA


Nun können wir die Spalte mit den dreistelligen Länder-Codes hinzufügen. Dazu nutzen wir die vorher bereits importierte `ISO3166`-Liste.

In [46]:
data['GeoId3'] = data['GeoId'].replace(iso3166.set_index('ISO3166-ALPHA-2')['ISO3166-ALPHA-3'])
data.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,GeoId3
0,2020-03-24,24,3,2020,6,1,Afghanistan,AF,AFG
1,2020-03-23,23,3,2020,10,0,Afghanistan,AF,AFG
2,2020-03-22,22,3,2020,0,0,Afghanistan,AF,AFG
3,2020-03-21,21,3,2020,2,0,Afghanistan,AF,AFG
4,2020-03-20,20,3,2020,0,0,Afghanistan,AF,AFG


Wir wir hier sehen können, wurde die benötigte Spalte `GeoId3` hinzugefügt und mit den entsprechenden dreistelligen Länder-Codes befüllt.

Da sowohl die Index-Spalte als auch die Kopfzeile der Tabelle bereits vielversprechend formatiert sind und es keine weiteren Daten gibt, die bereinigt werden müssen, können wir den Datensatz so weiter verwenden. 

# Anwendung - Statistik

##  Tagesdaten für ein Land

Beispielsweise können wir die Daten in kleinere Einheiten aufteilen, um nicht zu jeder Zeit mit dem gesamten Datensatz arbeiten zu müssen. Über Ansprechen der `GeoId`-Spalte können wir die Daten für Deutschland hersausfiltern. Wir speichern die Daten entsprechend in der Variable `data_deu`.

In [36]:
def data_subset_country(geoid3):
    return data[data.GeoId3 == geoid3].reset_index(drop=True)

data_deu = data_subset_country("DEU")
data_deu.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,GeoId3
0,24/03/2020,24,3,2020,4438,32,Germany,DE,DEU
1,23/03/2020,23,3,2020,3311,27,Germany,DE,DEU
2,22/03/2020,22,3,2020,3276,22,Germany,DE,DEU
3,21/03/2020,21,3,2020,4049,2,Germany,DE,DEU
4,20/03/2020,20,3,2020,5940,30,Germany,DE,DEU


Hier sieht man der Verlauf der neu bestätigten Fälle und Todesfälle in Deutschland.

In [38]:
def plot_verlauf(data):    
    fig = make_subplots(rows=2, cols=1)

    fig.add_trace(
        go.Scatter(x=data.DateRep, y=data.Cases, name="Fälle"),
        row=1, col=1,
    )

    fig.add_trace(
        go.Scatter(x=data.DateRep, y=data.Deaths, name="Todesfälle"),
        row=2, col=1
    )
    
    fig.update_layout(title_text='Verlauf bestätigter Fälle und Todesfälle')

    fig.show()
    
plot_verlauf(data_deu)

Nun schauen wir, welche Daten ohne Weiteres aus der vorhandenen Tabell extrahiert werden können. Zum einen können wir die Summe der bestätigten Krankheitsfälle ausgeben lassen. Zum anderen lässt sich auch die Zahl der bestätigten Todesfälle extrahieren.

In [11]:
deu_cases = data_deu.Cases.sum()
deu_cases

29212

In [12]:
deu_deaths = data_deu.Deaths.sum()
deu_deaths

126

## Akkumulierte Daten

Um die Gesamtzahlen aller Fälle und Todesälle darstellen zu können, summieren wir die Tagesdaten auf und erhalten so eine Liste aller Länder und den aufsummierten Zahlen. Dies machen wir zum einen kumulativ, sodass wir einen Verlauf darstellen können und zum anderen erstellen wir ein Sub-Set `data_total`, welches nur die Summe notiert.

Dazu werden in beiden Datensätzen Verhältnis-Daten zwischen Todesfällen und Infizierten hinzugefügt.

In [13]:
data['CumCases'] = data.sort_values(by='DateRep').groupby('GeoId3')['Cases'].cumsum()
data['CumDeaths'] = data.sort_values(by='DateRep').groupby('GeoId3')['Deaths'].cumsum()
data['CumRatio'] = data['Deaths'] / data['Cases'] * 100

In [14]:
data_total = data.groupby(['GeoId3',
                           "Countries and territories"],
                          as_index=False).sum()[['GeoId3', 
                                                 'Countries and territories', 
                                                 'Cases', 
                                                 'Deaths']]

data_total['Ratio'] = data_total['Deaths'] / data_total['Cases'] * 100

In [15]:
def plot_gesamtzahlen(data=data_total, sort=True, limit=15):
    """
    Horizontales Balkendiagramm mit den Gesamtzahlen der Fälle und Todesfälle.
    
    data: Datensatz
    sort: Boolean; soll sortiert werden?
    limit: Boolean; sollen nur die 'top' Einträge angezeigt werden? (Nur in Verbindung mit sort zu verwenden)
    """
    
    if sort:
        if limit:
            data = data.sort_values('Cases').tail(limit)
        else:
            data = data.sort_values('Cases')
            
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=data['Cases'],
        y=data['Countries and territories'],
        orientation='h',
        name='Infizierte',
        marker_color="green"))
    
    fig.add_trace(go.Bar(
        x=data['Deaths'],
        y=data['Countries and territories'],
        orientation='h',
        name='Todesfälle',
        marker_color="red"))
     
    fig.update_layout(barmode='stack', xaxis_tickangle=-45, title_text='Bestätigte Fälle und Todesfälle')

    fig.show()
    
def plot_ratio(data=data_total, sort=True, limit=15):
    
    if sort:
        if limit:
            data = data.sort_values('Cases').tail(limit)
        else:
            data = data.sort_values('Cases')
                
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=data['Ratio'],
        y=data['Countries and territories'],
        orientation='h',
        marker_color="blue"))
    
    fig.update_layout(xaxis_tickangle=-45, title_text='Verhältnis von Todesfällen zu Infizierten in %')

    fig.show()
    
def plot_verlauf_cum(data):
    fig = make_subplots(rows=3, cols=1)

    fig.add_trace(
        go.Scatter(x=data.DateRep, y=data['CumCases'], name="Fälle"),
        row=1, col=1,
    )

    fig.add_trace(
        go.Scatter(x=data.DateRep, y=data['CumDeaths'], name="Todesfälle"),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=data.DateRep, y=data['CumRatio'], name="Verhältnis"),
        row=3, col=1
    )
    
    country = data['Countries and territories'][0]
    fig.update_layout(title_text='Verlauf bestätigter Fälle, Todesfälle und deren Verhältnis - {}'.format(country))

    fig.show()

In [16]:
plot_gesamtzahlen(data_total)

Hierbei ist das Verhältnis besonders auffällig, weshalb wir dieses einmal alleine plotten; jedoch sortiert weiterhin nach den Infizierten-Zahlen.

In [17]:
plot_ratio()

# Anwendung Weltkarten

Für die Darstellung auf Weltkarten, schreiben wir eine Funktion, die einen Datensatz als Input nimmt und die entsprechende Karte ausgibt. Über den Wert von `plot_value` lässt sich angeben, ob die Inifizierten-Fälle oder die Todesfälle angezeigt werden sollen.

In [18]:
def plot_world_map(data=data, plot_value='Cases'):
    """
    data: Datensatz, der geplottet werden soll
    plot_value: 'Cases' oder 'Deaths'
    """
    
    world_map = go.Figure(data=go.Choropleth(
        z = data[plot_value],
        locations = data['GeoId3'],
        colorscale = 'Greens' if plot_value == 'Cases' else 'Reds' if plot_value == 'Deaths' else 'Blues',
        marker_line_width=0.2,
        colorbar_title = plot_value,
        text = data['Countries and territories'],
    ))
    
    world_map.update_layout(geo=dict(showframe=False, showcoastlines=False, projection_type='equirectangular'))
    world_map.show()

## Bestätigte Infizierte

In [19]:
plot_world_map(data_total, 'Cases')

## Bestätige Todesfälle

In [20]:
plot_world_map(data_total, 'Deaths')

## Verhältnis der Todesfälle zu den Infiziertenzahlen

In [21]:
plot_world_map(data_total, plot_value='Ratio')

In [27]:
plot_verlauf_cum(data_subset_country('DEU'))

# Ausblick

Möglichkeiten, das Projekt zu erweitern:

- *Timeline*: Weltkarte mit Zeitleiste, um den Verlauf der bestätigten Fälle beobachten zu können. Dies ist jedoch mit den genutzten Programmen nicht möglich. Plotly bietet keine Zeitleisten für Weltkarten an. Sollte sich ein entsprechendes Programm finden, müsste der Datensatz so bearbeitet werden, dass für jeden Tag ein Eintrag für jedes Land existiert. Außerdem müssen die Zeilen im Datensatz immer die akkumulierten Zahlen zeigen; nicht die *an dem Tag* bestätigten.
- *Weitere Datensätze*: Der Datensatz des ECDC beschränkt sich auf wesentliche geografische Informationen. Eine Aufschlüsselung in kleinere geografische Einheiten war mir nicht möglich. Das Robert-Koch-Institut besitzt solche Daten, macht diese jedoch nicht für die Allgemeinheit zugänglich. Außerdem wäre interessant, ein Datensatz zu nutzen, der über die geografischen Daten hinaus auch Personendaten umfasst. Dabei ist das Alter der PatientInnen vermutlich besonders interessant.